### This tutorial presents how to simulate the full network with time dependent dynamic input currents

## Step 1: Import necessary packages and modules

In [ ]:
# This line allows you to display matplotlib plots within the Jupyter Notebook
%matplotlib inline

# Import External packages 

import os
import numpy as np
import matplotlib.pyplot as plt

# Configure the working directory (Important: This should be set to home directory of 'dynworm' folder)

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

# Import Main module

import dynworm as dw

## Step 1: Initialize neural parameters and connectivity

In [ ]:
dw.network_sim.initialize_params_neural()
dw.network_sim.initialize_connectivity()

## Step 2: Define dynamic input stimuli
### The dynamic stimuli should be in the form of numpy.array of shape (timesteps x 279)
### Each column defines the time dependent inputs into i-th neuron
### In this tutorial, we will use in-built functions from dynworm.utils for sinusoidal and noisy inputs
### You can also construct your own dynamic stimuli with custom functions

In [ ]:
# Use dw.utils.neuron_names_2_inds to find the indices of the neurons

print(dw.utils.neuron_names_2_inds(['PLML']), dw.utils.neuron_names_2_inds(['PLMR']))

### dw_utils.construct_dyn_inputmat takes following parameters
### t0: Starting time point
### tf: Ending time point 
### input_type: 'sinusoidal' or 'noisy'
### neuron_indices: List of neurons' indices to be stimulated
### normalized_amps:
### i) with input_type = 'sinusoidal', list of amplitudes of sine waves. 
### ii) with input_type = 'noisy', list of base input amplitudes without noises
### freqs: xclusive to input_type = 'sinusoidal'. Determines frequencies of sine wave inputs
### noise_amplitude: exclusive to input_type = 'noisy'. Determines the noise amplitudes of noisy inputs

In [ ]:
input_mat_sinusoidal = dw.utils.construct_dyn_inputmat(t0=0, tf=21, input_type='sinusoidal', \
                                                       neuron_indices=[276, 278], 
                                                       normalized_amps=[2000, 3000], \
                                                       freqs=[3, 4])

In [ ]:
input_mat_noisy = dw.utils.construct_dyn_inputmat(t0=0, tf=21, input_type='noisy', \
                                                  neuron_indices=[276, 278], 
                                                  normalized_amps=[2000, 3000], 
                                                  noise_amplitudes=[50, 50])

### The figure below shows the different dynamic input into PLMR

In [ ]:
fig = plt.figure(figsize=(9, 12))

plt.subplot(211)
plt.title('Sinusoidal input')
plt.plot(input_mat_sinusoidal[:, 276], color = 'black')

plt.subplot(212)
plt.title('Noisy input')
plt.plot(input_mat_noisy[:, 276], color = 'black')

## Step 3: Configure ablation mask array

In [ ]:
ablation_mask = np.ones(dw.network_sim.params_obj_neural['N'], dtype = 'bool')

## Step 4: Run the simulation
### run_network_dyninput simulates takes following parameterss
### input_mat: the input_mat array with configured time dependent inputs (timepoints x 279)
### ablation_mask: the ablation vector which define any neurons to be ablated

In [ ]:
result_dict_sinusoidal = dw.network_sim.run_network_dyninput(input_mat=input_mat_sinusoidal, 
                                                  ablation_mask = ablation_mask)

In [ ]:
result_dict_noisy = dw.network_sim.run_network_dyninput(input_mat=input_mat_noisy, 
                                                  ablation_mask = ablation_mask)

## Step 5: Extract the result
### run_network_dyninput outputs an identical dictionary object as run_network_constinput from tutorial 1

In [ ]:
result_dict_sinusoidal.keys()

### In this example, we will take a look at the 'v_solution' 
### We take transpose to change the shape to spatial neurons x temporal axis 

In [ ]:
v_sol = result_dict_sinusoidal['v_solution'].T

### One can use matplotlib.pyplot.pcolor to visualize the activities of all neurons 

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[:, 100:], cmap='bwr', vmin = -20, vmax = 20)
plt.colorbar()

### Or only the motorneuronal activities

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[dw.neurons_idx.motor_group, 100:-100], cmap='bwr')
plt.colorbar()

### Here we show the activities of Ventral Type B, D and Dorsal Type B, D neurons

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,1)
plt.pcolor(v_sol[dw.neurons_idx.VB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,2)
plt.pcolor(v_sol[dw.neurons_idx.VD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VD_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,3)
plt.pcolor(v_sol[dw.neurons_idx.DB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,4)
plt.pcolor(v_sol[dw.neurons_idx.DD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DD_ind), 0)
plt.colorbar()